## Part 4. Dashboard Application using Plotly Dash

In [1]:
# Import required libraries
import pandas as pd
from dash import Dash, html,dcc,callback
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',  options=[{'label': 'All Sites', 'value': 'ALL'},
                                                                           {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                                           {'label':'CCAFS SLC-40', 'value':'CCAFS SLC-40'},
                                                                           {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                                           {'label':'VAFB SLC-4E', 'value':'VAFB SLC-4E'}],
                                                                  value='ALL', placeholder="Select a Launch Site here",
                                                                  searchable=True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min = 0,
                                                max = 10000,
                                                step = 1000,
                                                value = [min_payload,max_payload]
                                               ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown',component_property='value'))
def get_pie_chart(value):
    filtered_df = spacex_df.groupby('Launch Site',as_index=False).sum()
    if value == 'ALL':
        fig = px.pie(filtered_df, values='class', 
        names='Launch Site', 
        title='Total Success Launches by Site')
        return fig
    else:
        df_1= spacex_df[spacex_df['Launch Site']==value]
        count_df = df_1['class'].value_counts().reset_index()
        count_df.columns = ['Value','Count']
        # Create a pie chart using Plotly Express
        fig = px.pie(count_df, values='Count', names='Value', title='Total Success Launches for site {}'.format(value))
        # return the outcomes piechart for a selected site
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id = 'success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown',component_property='value'),
               Input(component_id = "payload-slider", component_property="value")])

def get_scatter_chart(dropdown_value,slider_range):
    low,high = slider_range
    data = spacex_df[['Launch Site','Payload Mass (kg)','class','Booster Version Category']]
    data_slider = data[(data['Payload Mass (kg)']>low) & (data['Payload Mass (kg)']<high)]
    
    if dropdown_value == 'ALL':
        fig = px.scatter(data_slider,x='Payload Mass (kg)', y='class',color= 'Booster Version Category',
                         title='Correlation between Payload and Success for all sites')
        return fig
    else:
        data_selected = data_slider[data_slider['Launch Site']==dropdown_value]
        fig = px.scatter(data_selected, x='Payload Mass (kg)', y='class',color='Booster Version Category',
                        title='Correlation between Payload and Success for {}'.format(dropdown_value))
        return fig
# Run the app
if __name__ == '__main__':
    app.run_server()
